![image](https://scontent.fcdg1-1.fna.fbcdn.net/v/t1.15752-9/89548024_216782202724810_4226178270858575872_n.png?_nc_cat=102&_nc_sid=b96e70&_nc_oc=AQmslKDI-sSzwE3UmHVUwK4goO4ImZ0D497D1AnJkMKyTvP-4x8C8l5LdbhfWg9vsiM&_nc_ht=scontent.fcdg1-1.fna&oh=b27c5abe3f1ce0620a0008835bd0113a&oe=5E8F633F)

**PREPARATION DE L'ENVIRONNEMENT**

In [0]:
#Verification version tensorflow. Il faut que ça soit 1.x
from tensorflow.python.framework.versions import VERSION
VERSION

'1.15.0'

In [0]:
#Installation du package nécessaire
!pip install face_recognition
#!pip install tensorflow==1.5.0  #A lancer seulement si la version de tensorflow est 2.x

In [0]:
import face_recognition as fr
import cv2
import numpy as np
import os
import glob
from google.colab.patches import cv2_imshow
from time import sleep
import imutils
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers
from keras.regularizers import l1
from keras.models import model_from_json
#Import Drive
from google.colab import drive
drive.mount('/content/drive')

**TELECHARGER LE MODELE**

In [0]:
#Si le dossier faSISE-RiccoGnition est bien ajouté à votre GDrive, le lien devrait être inchangé
model = model_from_json(open("/content/drive/My Drive/faSISE-RiccoGnition/facial_expression_model_structure.json", "r").read())
model.load_weights('/content/drive/My Drive/faSISE-RiccoGnition/facial_expression_model_weights.h5') #load weights

**PREPARER LE MODELE**

In [0]:
#La fonction pour recoder toutes les images en vector
def get_encoded_faces():
    """
    looks through the faces folder and encodes all
    the faces

    :return: dict of (name, image encoded)
    """
    encoded = {}

    for dirpath, dnames, fnames in os.walk("/content/drive/My Drive/faSISE-RiccoGnition/photo_train"):
        for f in fnames:
            if f.endswith(".JPG") or f.endswith(".png"):
                face = fr.load_image_file("/content/drive/My Drive/faSISE-RiccoGnition/photo_train/" + f)
                encoding = fr.face_encodings(face)[0]
                encoded[f.split(".")[0]] = encoding

    return encoded

# Cette action transforme l'intégralité des photos train. Cela peut prendre quelques minutes.
faces = get_encoded_faces()

In [0]:
#La fonction pour predire l'emotion
def emotion(im, face_location, model=model):
 top, right, bottom, left = face_location
 face_image = im[top:bottom, left:right]
 face_image = cv2.resize(face_image, (48,48))
 face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
 face_image=np.reshape(face_image,[48,48,1])
 #cv2_imshow(face_image)
 face_image = np.expand_dims(face_image, axis = 0)
 
 predictions = model.predict(face_image)
 
 #find max indexed array
 max_index = np.argmax(predictions[0])
 emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
 emotion = emotions[max_index] 
 return emotion

In [0]:
#La fonction pour encoder une image à partir d'un nom de fichier (utilisée pour les new-data)
def unknown_image_encoded(img):
    """
    encode a face given the file name
    """
    face = fr.load_image_file("/content/drive/My Drive/faSISE-RiccoGnition/photo_train/" + img)
    encoding = fr.face_encodings(face)[0]

    return encoding

emotions_color={'angry':(178,34,34), 'disgust':(107,142,35), 'fear':(255,140,0), 'happy':(72,209,204), 'sad':(105,105,105), 'surprise':(106,90,205), 'neutral':(192,192,192)}

#La fonction pour comparer les visages
def classify_face(im,faces=faces):
    """
    will find all of the faces in a given image and label
    them if it knows what they are

    :param im: str of file path
    :return: list of face names
    """
    faces_encoded = list(faces.values())
    known_face_names = list(faces.keys())

    img = cv2.imread(im, 1)
    #img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
    #img = img[:,:,::-1]
 
    face_locations = fr.face_locations(img)
    nb_faces = len(face_locations)
    unknown_face_encodings = fr.face_encodings(img, face_locations)

    face_names = []
    emotions = []
    i = 0
    for face_encoding in unknown_face_encodings:

        #### Reconnaissance emotion
        emot = emotion(img, face_locations[i])
        emotions.append(emot)
        i+=1
        #print(emot)

        ### Reconnaissance id
        # See if the face is a match for the known face(s)
        matches = fr.compare_faces(faces_encoded, face_encoding)
        name = "Unknown"

        # use the known face with the smallest distance to the new face
        face_distances = fr.face_distance(faces_encoded, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    i = 0
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a box around the face
        cv2.rectangle(img, (left-30, top-30), (right+30, bottom+65), emotions_color[emotions[i]], 2)

        # Draw a label with a name below the face
        cv2.rectangle(img, (left-30, bottom -30), (right+30, bottom+65), emotions_color[emotions[i]], cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, name, (left -20, bottom + 15), font, 1.5, (255, 255, 255), 2)
        cv2.putText(img, emotions[i], (left -20, bottom + 50), font, 1.5, (255, 255, 255), 2)
        i+=1

      
    # Display the resulting image
    imS = imutils.resize(img,width=1200)
    #cv2_imshow(imS)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    return imS 


**TEST**

In [0]:
#Tester avec une photo
img = classify_face("/content/drive/My Drive/faSISE-RiccoGnition/photo_test/cally_caballero_nguyen.jpg")
cv2_imshow(img)

**MISE EN PRODUCTION**

In [0]:
#Installer les packages nécessaires
!pip install flask-ngrok
!pip install flask==0.12.2

In [0]:
#Importer les packages nécessaires
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import send_from_directory
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
import random

In [0]:
#Quelques mesures de sécurité sur l'upload des fichiers
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

app = Flask(__name__)

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
         

In [0]:
#Créer l'application et mis en ligne
#Lien de l'application web est de type: http://xxxxxxxx.ngrok.io se trouve en pas de la cellule.
#Tant que cette cellule fonction, l'application est en ligne. A chaque relance, le lien change.
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/", methods=['GET', 'POST'])
def riccognition():
    html_img = ""
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(filename)
            image = classify_face(filename)
            cv2.imwrite("predict.jpg", image) 
            rand = random.randint(1, 999999) # used to force browser to refresh image
            html_img =  '''<img class="p-5 bg-white shadow rounded-lg" src="predict.jpg?'''+str(rand)+'''" />'''
            #return redirect("predict.jpg") 
    return '''<!doctype html><title>faSise RiccoGnition</title><section> <div class="container p-5"> <div class="row mb-5 text-center text-white" style="margin-bottom: 0 !important;"> <div class="col-lg-10 mx-auto">
    <h1 class="display-4">faSise RiccoGnition </h1>
    <p class="lead">Reconnaissance faciale - SISE2020</p>
    </div></div><div class="row"> <div class="col-lg-5 mx-auto" style="max-width: unset"> <div class="p-5 bg-white shadow rounded-lg">
    <img src="https://scontent-cdg2-1.xx.fbcdn.net/v/t1.15752-9/89566114_2224862024283061_493235809504198656_n.png?_nc_cat=107&_nc_sid=b96e70&_nc_oc=AQmsbwknU4HP_bikq7wkVWhV73bwF7nrLTPr9LG-wc9oULBqH7l0hGyBoS8eXQASjvk&_nc_ht=scontent-cdg2-1.xx&oh=782ed0d24d64441f19d7b010c55c0537&oe=5E8ED5DE" alt="" width="400" class="d-block mx-auto mb-4 rounded-pill">
    <h6 class="text-center mb-4 text-muted"> Upload your picture for riccognition </h6>
    <form method=post enctype=multipart/form-data>
      <div class="custom-file overflow-hidden rounded-pill mb-5" style="margin-bottom: 0!important;">
      <input id="customFile" type="file" class="custom-file-input rounded-pill" name=file>
      <label for="customFile" class="custom-file-label rounded-pill">Choisir fichier ...</label>
      </div>
      <label for="fileUpload" class="file-upload btn btn-primary btn-block rounded-pill shadow" style="max-height: 40px;"><i class="fa fa-upload mr-2"></i>Lancer la reconnaissance
        <input id="fileUpload" type=submit class="custom-file-input rounded-pill" style="height: 0;"></label>
    </form>
    </div></div></div>              
    <div style="margin-top: 10px">'''+html_img+'''</div>
    </div></section><style>.file-upload input[type='file']{display: none;}body{background: #00B4DB; background: -webkit-linear-gradient(to right, #0083B0, #00B4DB); background: linear-gradient(to right, #0083B0, #00B4DB); height: 100vh;}.rounded-lg{border-radius: 1rem;}.custom-file-label.rounded-pill{border-radius: 50rem;}.custom-file-label.rounded-pill::after{border-radius: 0 50rem 50rem 0;}</style><link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" crossorigin="anonymous"><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/4.3.1/css/bootstrap.min.css" crossorigin="anonymous"><script src="https://code.jquery.com/jquery-3.3.1.slim.min.js" crossorigin="anonymous"></script><script src="https://cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/4.3.1/js/bootstrap.bundle.min.js" crossorigin="anonymous"></script>'''

#    return '''<!doctype html>
#      <title>Upload new File</title>
#      <h1>faSise RiccoGnition</h1>
#      <img src="https://scontent-cdg2-1.xx.fbcdn.net/v/t1.15752-9/89566114_2224862024283061_493235809504198656_n.png?_nc_cat=107&_nc_sid=b96e70&_nc_oc=AQmsbwknU4HP_bikq7wkVWhV73bwF7nrLTPr9LG-wc9oULBqH7l0hGyBoS8eXQASjvk&_nc_ht=scontent-cdg2-1.xx&oh=782ed0d24d64441f19d7b010c55c0537&oe=5E8ED5DE" alt="face recognition" />
#      <h2>Upload new File</h2>
#      <form method=post enctype=multipart/form-data>
#        <input type=file name=file>
#        <input type=submit value=Upload>
#      </form>
#      '''+html_img

@app.route('/<filename>')
def uploaded_file(filename):
    return send_from_directory("",filename)

if __name__ == '__main__':
  app.run()